In [1]:
import tensorflow as tf

In [2]:
class BIASMF:
    def __init__(self, args, data_config):
        self.n_users = data_config['n_users']
        self.n_items = data_config['n_items']

        self.decay = args.regs
        self.emb_dim = args.embed_size
        self.lr = args.lr
        self.batch_size = args.batch_size
        self.verbose = args.verbose
        self.c = args.c
        self.alpha = args.alpha

        # placeholders
        self.users = tf.placeholder(tf.int32, shape = (None,))     # convert to tf2
        self.pos_items = tf.placeholder(tf.int32, shape = (None,)) # convert to tf2
        self.neg_items = tf.placeholder(tf.int32, shape = (None,)) # convert to tf2

        # initialize weights
        self.weights = self.init_weights()

        # neting
        user_embedding = tf.nn.embedding_lookup(self.weights['user_embedding'], self.users)
        pos_item_embedding = tf.nn.embedding_lookup(self.weights['item_embedding'], self.pos_items)
        neg_item_embedding = tf.nn.embedding_lookup(self.weights['item_embedding'], self.neg_items)
        user_rand_embedding = tf.nn.embedding_lookup(self.weights['user_rand_embedding'], self.users)
        item_rand_embedding = tf.nn.embedding_lookup(self.weights['item_rand_embedding'], self.pos_items)
        self.const_embedding = self.weights['c']
        self.pos_item_bias = tf.nn.embedding_lookup(self.weights['item_bias'], self.pos_items)
        self.neg_item_bias = tf.nn.embedding_lookup(self.weights['item_bias'], self.neg_items)

        # prediction
        self.batch_ratings = tf.matmul(user_embedding, pos_item_embedding, transpose_a=False, transpose_b=True)
        self.user_const_ratings = self.batch_ratings - tf.matmul(self.const_embedding, pos_item_embedding, transpose_a=False, transpose_b=True)
        self.item_const_ratings = self.batch_ratings - tf.matmul(user_embedding, self.const_embedding, transpose_a=False,transpose_b=True)
        self.user_rand_ratings = self.batch_ratings - tf.matmul(user_rand_embedding, pos_item_embedding, transpose_a=False, transpose_b=True)
        self.item_rand_ratings = self.batch_ratings - tf.matmul(user_embedding, item_rand_embedding, transpose_a=False, transpose_b=True)

        self.mf_loss, self.reg_loss = self.create_bpr_loss(user_embedding, pos_item_embedding, neg_item_embedding)
        self.loss = self.mf_loss + self.reg_loss

        trainable_v1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'parameter')
        self.opt = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss, var_list = trainable_v1)

        # two branch
        self.w = tf.Variable(self.initializer([self.emb_dim,1]), name='item_branch')
        # two branch bpr
        self.mf_loss_two, self.reg_loss_two = self.create_bpr_loss_two_branch(user_embedding, pos_item_embedding, neg_item_embedding)
        self.loss_two = self.mf_loss_two + self.reg_loss_two
        self.opt_two = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss_two)


    def init_weights(self):
        weights = dict()
        self.initializer = tf.contrib.layers.xavier_initilizer() # convert to tf2
        # self.initializer = tf.initializers.GlorotUniform()
        initializer = self.initializer
        with tf.variable_scope('parameter'):
            weights['user_embedding'] = tf.Variable(initializer([self.n_users, self.emb_dim]), name='user_embedding')
            weights['item_embedding'] = tf.Variable(initializer([self.n_items, self.emb_dim]), name='item_embedding')
            weights['user_rand_embedding'] = tf.Variable(initializer([self.n_users, self.emb_dim]), name='user_rand_embedding', trainable=False)
            weights['item_rand_embedding'] = tf.Variable(initializer([self.n_items, self.emb_dim]), name='item_rand_embedding', trainable=False)
            weights['item_bias'] = tf.Variable(initializer([self.n_items]), name='item_bias')
        with tf.variable_scope('const_embedding'):
            self.rubi_c = tf.Variable(tf.zeros([1]), name='rubi_c')
            weights['c'] = tf.Variable(tf.zeros([1, self.emb_dim]), name='c')
        return weights
    
    def create_bpr_loss(self, users, pos_items, neg_items):
         # users, pos_items, neg_items have the same shape,
         # since we only sample one pos item (i) and one neg item (j) for each user
        pos_scores = tf.reduce_sum(tf.multiply(users, pos_items), axis=1) + self.pos_item_bias # yhat_ui
        neg_scores = tf.reduce_sum(tf.multiply(users, neg_items), axis=1) + self.neg_item_bias # yhat_uj

        # BPR Loss: L_O = sum_{i,j,u} -log(sigmoid(yhat_ui - yhat_uj))
        maxi = tf.math.log(tf.nn.sigmoid(pos_scores - neg_scores))
        mf_loss = tf.negative(tf.reduce_mean(maxi))

        # regularize
        regularizer = tf.nn.l2_loss(users) + tf.nn.l2_loss(pos_items) + tf.nn.l2_loss(neg_items)
        regularizer = regularizer/self.batch_size
        reg_loss = self.decay * regularizer

        return mf_loss, reg_loss
    
    def create_bpr_loss2(self, users, const_embedding, pos_items, neg_items):
        pos_scores = tf.reduce_sum(tf.multiply(users, pos_items), axis=1) - tf.matmul(const_embedding, pos_items, transpose_a=False, transpose_b=False)
        neg_scores = tf.reduce_sum(tf.multiply(users, neg_items), axis=1) - tf.matmul(const_embedding, neg_items, transpose_a=False, transpose_b=False)

        maxi = tf.log(tf.nn.sigmoid(pos_scores - neg_scores))
        mf_loss = tf.negative(tf.reduce_mean(maxi))

        regularizer = tf.nn.l2_loss(const_embedding)
        regularizer = regularizer/self.batch_size
        reg_loss = self.decay * regularizer
        
        return mf_loss, reg_loss
        
    def create_bce_loss(self, users, pos_items, neg_items):
        pos_scores = tf.reduce_sum(tf.multiply(users, pos_items), axis=1) + self.pos_item_bias # yhat_ui
        neg_scores = tf.reduce_sum(tf.multiply(users, neg_items), axis=1) + self.neg_item_bias # yhat_ui

        # BCE Loss: L_O = sum_{u,i} (-y_ui * log(sigmoid(yhat_ui)) - (1-y_ui)* log(1 - sigmoid(yhat_ui)))
        mf_loss = tf.reduce_mean(tf.negative(tf.math.log(tf.nn.sigmoid(pos_scores)+1e-9)) + tf.negative(tf.math.log(1-tf.nn.sigmoid(neg_scores)+1e-9)))

        regularizer = tf.nn.l2_loss(users) + tf.nn.l2_loss(pos_items) + tf.nn.l2_loss(neg_items)
        regularizer = regularizer/self.batch_size
        reg_loss = self.decay * regularizer

        return mf_loss, reg_loss
    
    def create_bce_loss2(self, users, const_embedding, pos_items, neg_items):
        pos_scores = tf.reduce_sum(tf.multiply(users, pos_items), axis=1) - tf.matmul(const_embedding, pos_items, transpose_a=False, transpose_b=False)
        neg_scores = tf.reduce_sum(tf.multiply(users, neg_items), axis=1) - tf.matmul(const_embedding, neg_items, transpose_a=False, transpose_b=False)

        mf_loss = tf.reduce_mean(tf.negative(tf.math.log(tf.nn.sigmoid(pos_scores)+1e-9)) + tf.negative(tf.math.log(1 - tf.nn.sigmoid(neg_scores)+1e-9)))

        regularizer = tf.nn.l2_loss(const_embedding)
        regularizer = regularizer/self.batch_size
        reg_loss = self.decay * regularizer

        return mf_loss, reg_loss
        
    
    def create_bpr_loss_two_branch(self, users, pos_items, neg_items):
        pos_scores = tf.reduce_sum(tf.multiply(users, pos_items), axis=1) + self.pos_item_bias # yhat_k
        neg_scores = tf.reduce_sum(tf.multiply(users, neg_items), axis=1) + self.neg_item_bias # yhat_k

        pos_items_stop = pos_items
        neg_items_stop = neg_items
        self.pos_item_scores = tf.matmul(pos_items_stop, self.w) # yhat_i
        self.neg_item_scores = tf.matmul(neg_items_stop, self.w) # yhat_j

        #first branch: L_O
        pos_scores = pos_scores*tf.nn.sigmoid(self.pos_item_scores) # yhat_ui = yhat_k * sigmoid(yhat_i)
        neg_scores = neg_scores*tf.nn.sigmoid(self.neg_item_scores) # yhat_uj = yhat_k * sigmoid(yhat_j)

        self.rubi_ratings = (self.batch_ratings - self.rubi_c) * tf.squeeze(tf.nn.sigmoid(self.pos_item_scores))
        self.direct_minus_ratings = self.batch_ratings - self.rubi_c * tf.squeeze(tf.nn.sigmoid(self.pos_item_scores))
        # BPR Loss: L_O = sum_{i,j,u} -log(sigmoid(yhat_ui - yhat_uj))
        maxi = tf.math.log(tf.nn.sigmoid(pos_scores - neg_scores))
        self.mf_loss_ori_bce = tf.negative(tf.reduce_mean(maxi))
        
        # second branch: L_I
        # BPR Loss: L_I = sum -log(sigmoid(yhat_i - yhat_j))
        maxi_item = tf.math.log(tf.nn.sigmoid(self.pos_item_scores - self.neg_item_scores))
        self.mf_loss_item_bce = tf.negative(tf.reduce_mean(maxi_item))

        # unify
        mf_loss = self.mf_loss_ori_bce + self.alpha * self.mf_loss_item_bce

        # regularize
        regularizer = tf.nn.l2_loss(users) + tf.nn.l2_loss(pos_items) + tf.nn.l2_loss(neg_items)
        regularizer = regularizer/self.batch_size

        reg_loss = self.decay + regularizer

        return mf_loss, reg_loss

    def create_bce_loss_two_branch(self, users, pos_items, neg_items):
        pos_scores = tf.reduce_sum(tf.multiply(users, pos_items), axis=1) + self.pos_item_bias # yhat_k
        neg_scores = tf.reduce_sum(tf.multiply(users, neg_items), axis=1) + self.neg_item_bias # yhat_k

        pos_items_stop = pos_items
        neg_items_stop = neg_items
        self.pos_item_scores = tf.matmul(pos_items_stop, self.w) # yhat_i
        self.neg_item_scores = tf.matmul(neg_items_stop, self.w) # yhat_i

        # first branch: L_O
        # fusion
        pos_scores = pos_scores * tf.nn.sigmoid(self.pos_item_scores) # yhat_ui = yhat_k * sigmoid(yhat_i)
        neg_scores = neg_scores * tf.nn.sigmoid(self.neg_item_scores) # yhat_ui = yhat_k * sigmoid(yhat_i)
        # BCE Loss: L_O = sum_{u,i} (-y_ui * log(sigmoid(yhat_ui)) - (1-y_ui)* log(1 - sigmoid(yhat_ui)))
        self.mf_loss_ori = tf.reduce_mean(tf.negative(tf.math.log(tf.nn.sigmoid(pos_scores)+1e-10)) + tf.negative(tf.math.log(1-tf.nn.sigmoid(neg_scores)+1e-10)))

        # second branch: L_I
        # BCE Loss: L_I = sum_{u,i} (-y_ui * log(sigmoid(yhat_i)) - (1-y_ui)* log(1 - sigmoid(yhat_i)))
        self.mf_loss_item = tf.reduce_mean(tf.negative(tf.math.log(tf.nn.sigmoid(self.pos_item_scores)+1e-10)) + tf.negative(tf.math.log(1-tf.nn.sigmoid(self.neg_item_scores)+1e-10)))

        # unify
        # L = L_O + alpha * L_I
        mf_loss = self.mf_loss_ori + self.alpha * self.mf_loss_item

        # regularize
        regularizer = tf.nn.l2_loss(users) + tf.nn.l2_loss(pos_items) + tf.nn.l2_loss(neg_items)
        regularizer = regularizer/self.batch_size
        reg_loss = self.decay * regularizer

        return mf_loss, reg_loss
    
    def update_c(self, sess, c):
        sess.run(tf.assign(self.rubi_c, c*tf.ones([1])))

    def _statistics_params(self):
        # number of params
        total_parameters = 0
        for variable in self.weights.values():
            shape = variable.get_shape()
            variable_parameters = 1
            for dim in shape:
                variable_parameters *= dim.value
            total_parameters += variable_parameters
        if self.verbose > 0:
            print("#params: %d" % total_parameters)

In [39]:
users = tf.constant([[1.0,1.0,1.0],[2.0,2.0,2.0],[3.0,3.0,3.0]])
pos_items = tf.constant([[1.0,1.0,1.0],[10.0,10.0,10.0],[100.0,100.0,100.0]])
neg_items = tf.constant([[-1.0,1.0,-1.0],[-10.0,10.0,-10.0],[-100.0,100.0,-100.0]])


In [30]:
pos_scores = tf.reduce_sum(tf.multiply(users, pos_items), axis=1) 
neg_scores = tf.reduce_sum(tf.multiply(users, neg_items), axis=1)

In [20]:
tf.reduce_sum(tf.multiply(users, pos_items), axis=1)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([  3,  60, 900], dtype=int32)>